In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model

import os

In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image

In [3]:
TARGET_SIZE = (100, 100)

## Data Augmentation 

In [4]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.2, 1.5]
)

### Generator of train images

In [5]:
data = os.path.join(os.getcwd(), 'dataset/processed_split/train')

train_generator = datagen.flow_from_directory(   
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(train_generator)
print("Number of batches: ", num_batches)

Found 1280 images belonging to 16 classes.
Number of batches:  40


### Generator of test images

In [6]:
data = os.path.join(os.getcwd(), 'dataset/processed_split/test')

test_generator = datagen.flow_from_directory(
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(test_generator)
print("Number of batches: ", num_batches)

Found 320 images belonging to 16 classes.
Number of batches:  10


Son 100 imagenes, de 100x100, en los 3 canales RGB.

## Modelo de Datos

In [7]:
base_filters = 32
w_regularizer = 1e-4

In [8]:
model = tf.keras.Sequential()

# First layer
model.add(Conv2D(filters=base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu', input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Second layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# Third layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Fourth layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Fifth layer
model.add(Conv2D(filters=3*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Sixth layer
model.add(Conv2D(filters=3*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten the output
model.add(Flatten())
model.add(Dense(16, activation='softmax'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 32)      320       
                                                                 
 activation (Activation)     (None, 100, 100, 32)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 100, 100, 64)      18496     
                                                                 
 activation_1 (Activation)   (None, 100, 100, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 50, 50, 64)        0         
                                                        

In [9]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [10]:
# NOTE: We are adding 1 to the number of steps per epoch because of an unexpected behavior with the iterator
#      that causes it to skip every other epoch, therefore showing an OUT OF RANGE warning.
#      This is a temporary fix until the issue is resolved.
hist = model.fit(train_generator, steps_per_epoch=len(train_generator)+1, epochs=50, verbose=1)

Epoch 1/50
41/41 [==============================] - 4s 33ms/step - loss: 2.8340 - accuracy: 0.0531


In [11]:
# Assuming test_images and test_labels are your testing data and labels
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

10/10 [==============================] - 0s 21ms/step - loss: 2.8008 - accuracy: 0.0625

Test accuracy: 0.0625

Test loss: 2.8007848262786865


## Save model 

In [12]:
# Save the model
model.save('out/cnn2.keras')